# Imports

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import json
from string import punctuation
import pickle
import random
from tensorflow.keras.models import load_model

# Loading Data

In [68]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
intents = json.loads(open('intents.json').read())
model = load_model('chatbot_model.h5')

# Functions

In [7]:
lemmatizer = WordNetLemmatizer()

In [34]:
def cleanup_sentence(sentence: str) -> list:
    tokenized = nltk.word_tokenize(sentence)
    lemmatized = [lemmatizer.lemmatize(word.lower()) for word in tokenized]
    return lemmatized

In [36]:
def bag_of_words(words: list, sentence_words: list) -> np.array:
    bag = [0] * len(words)
    for sentence_word in sentence_words:
        for i, word in enumerate(words):
            if sentence_word == word:
                bag[i] = 1
    return np.array(bag)

In [171]:
def predict_class(msg: np.array) -> list:
    msg_class_probs = model.predict(np.array([msg]))[0]
    results = [[i,r] for i, r in enumerate(msg_class_probs)]
    results.sort(key=lambda x: x[1], reverse=True)
    return results


In [205]:
def get_response(msg_class: str) -> str:
    for tag in intents['intents']:
        if tag['tag'] == msg_class:
            random_response = random.choice(tag['responses'])
    return random_response

In [243]:
def main(msg: str):
    msg = cleanup_sentence(msg)
    msg = bag_of_words(words, msg)
    msg_class_probs = predict_class(msg)
    msg_class = classes[msg_class_probs[0][0]]
    response = get_response(msg_class)
    return response, msg_class

In [258]:
msg = 'I feel sad. tell me a joke'
response, msg_class = main(msg)
print(response)

1/1 [==============================] - 0s 30ms/step
The world tongue-twister champion just got arrested. I hear they're gonna give him a really tough sentence.
